In [7]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas
import heapq

In [10]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        # TODO
        # update visit_popularity
        # update purchase_popularity
        for id in visits.split(','):
            visit_popularity[id] += 1
        for id in purchases.split(','):
            purchase_popularity[id] += 1

In [14]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [39]:
def prec_K (rec, max_count, relevant, purch):
    rafp = 0
    for pp in purch:
        if (pp in rec) == True:
            rafp += 1
    return rafp/(float)(max_count), rafp/(float)(relevant)

In [65]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = purchases.split(',')
                # TODO
                # update avg_recall
                # update avg_precision
                # update sessions_count
                #print visits
                #print purchases
                for cc in range(max_count):
                    rec = recommend(visits, cc+1)
                    prAK,rAK = prec_K(rec, cc+1, len(purchases), purchases)
                    avg_recall[cc] += rAK
                    avg_precision[cc] += prAK
                    #print rec, prAK,rAK
                #print 'ROUND'
                sessions_count += 1
                
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')

In [66]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.80,0.69
2,0.53,0.84
3,0.39,0.89
4,0.31,0.91
5,0.25,0.93


In [67]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.53,0.46
2,0.38,0.64
3,0.30,0.73
4,0.25,0.79
5,0.21,0.82


In [68]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.51,0.44
2,0.38,0.63
3,0.30,0.73
4,0.25,0.79
5,0.21,0.82


In [69]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.48,0.42
2,0.36,0.60
3,0.29,0.70
4,0.24,0.76
5,0.20,0.80
